# Facebook Freunde Scrapen
- Eine super Datenquelle um Netzwerke zu verstehen sind die eigenen Freunde. 
- Diese lassen sich von Facebook scrapen, da Facebook natürlich keinen Export erlaubt - d'oh. 

In [12]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import csv
import re

In [13]:
s = webdriver.Chrome('/usr/local/bin/chromedriver')

https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.find_element_by_link_text- 

In [18]:
USERNAME = "aryanca@hotmail.com"# "#####@###.ch"
PASSWORD = "machee"
MY_SLUG = "elle.nora1" #"###.###"

In [19]:
def login(s):
    try:
        print("Logging into FB")
        s.get("http://facebook.com/login")
        s.find_element_by_id("email").send_keys(USERNAME)
        r = s.find_element_by_id("pass")
        r.send_keys(PASSWORD)
        r.send_keys(Keys.RETURN)
    except:
        print("Probably already logged in...")

In [20]:
def scrape_friends(s):
    login(s)
    #Scrape my friends
    s.get("https://www.facebook.com/%s/friends" % MY_SLUG)
    for i in range(30):
        print("scrolling...and...waiting..%s" % i)
        s.execute_script('window.scroll(0,100000);')
        time.sleep(1)
    # get html
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    friends = soup.select(".fcb a")
    print("Scraping {%s} friends" % len(friends))
    time.sleep(2)
    with open('friends.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["Friend ID", "Friend Name"])
        for friend in friends:
          try:
            friend_name = friend.text.encode("ascii","ignore")
            friend_id = json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")
            mycsv.writerow([friend_id, friend_name])
          except:
            print("Problems obtaining %s" % friend)

In [21]:
scrape_friends(s)

Logging into FB
scrolling...and...waiting..0
scrolling...and...waiting..1
scrolling...and...waiting..2
scrolling...and...waiting..3
scrolling...and...waiting..4
scrolling...and...waiting..5
scrolling...and...waiting..6
scrolling...and...waiting..7
scrolling...and...waiting..8
scrolling...and...waiting..9
scrolling...and...waiting..10
scrolling...and...waiting..11
scrolling...and...waiting..12
scrolling...and...waiting..13
scrolling...and...waiting..14
scrolling...and...waiting..15
scrolling...and...waiting..16
scrolling...and...waiting..17
scrolling...and...waiting..18
scrolling...and...waiting..19
scrolling...and...waiting..20
scrolling...and...waiting..21
scrolling...and...waiting..22
scrolling...and...waiting..23
scrolling...and...waiting..24
scrolling...and...waiting..25
scrolling...and...waiting..26
scrolling...and...waiting..27
scrolling...and...waiting..28
scrolling...and...waiting..29
Scraping {500} friends
Problems obtaining <a ajaxify="/ajax/friends/inactive/dialog?id=5270152

In [22]:
def scrape_mutual_friends(s):
    login(s)
    edgelist = []
    friendslist = []
    
    #Get FriendsIDs
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"],row["Friend Name"]])
                
    #Get my Id
    s.get("http://facebook.com/%s" % MY_SLUG)
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    my_id = re.search(r"profile_id=(\d*)", source_code).groups()[0]
    print("My ID: %s" % my_id)

    for friends in friendslist:
        print("Visiting friend: %s" % friends[1])
        s.get("https://www.facebook.com/browse/mutual_friends/?uid=%s&node=%s" % (my_id,friends[0]))
        elem = s.find_element_by_xpath("//*")
        source_code = elem.get_attribute("outerHTML")
        soup = BeautifulSoup(source_code, 'lxml')
        mutual_friends = soup.select(".fcb a")
        print("Found %s mutual friends" % len(mutual_friends))
        for friend in mutual_friends:
            edgelist.append([friends[0],json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")])

    with open('network.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["From","To"])
        for edge in edgelist:
            mycsv.writerow([edge[0],edge[1]])

In [23]:
scrape_mutual_friends(s)

Logging into FB
Probably already logged in...
My ID: 100000272199811
Visiting friend: b'Ella Cunliffe'
Found 29 mutual friends
Visiting friend: b'Kirsten Buckley'
Found 29 mutual friends
Visiting friend: b'Louise Simmons'
Found 29 mutual friends
Visiting friend: b'Nadja Smits'
Found 29 mutual friends
Visiting friend: b'Meret Aeppli'
Found 29 mutual friends
Visiting friend: b'Anthony Blargh'
Found 29 mutual friends
Visiting friend: b'Elisabete Subtil'
Found 29 mutual friends
Visiting friend: b'Pascal Scheiber'
Found 29 mutual friends
Visiting friend: b'Lukas Blatter'
Found 29 mutual friends
Visiting friend: b'Bernhard Aebersold'
Found 29 mutual friends
Visiting friend: b'Matthias Strasser'
Found 29 mutual friends
Visiting friend: b'Nina Mathys'
Found 29 mutual friends
Visiting friend: b'Julius E. O. Fintelmann'
Found 29 mutual friends
Visiting friend: b'Dominic Iseli'
Found 29 mutual friends
Visiting friend: b'Benjamin von Wyl'
Found 29 mutual friends
Visiting friend: b'Raphael Hnerfaut

Found 0 mutual friends
Visiting friend: b'Andr Behrendt'
Found 0 mutual friends
Visiting friend: b'Nolle Karpf'
Found 0 mutual friends
Visiting friend: b'Aline Wanner'
Found 0 mutual friends
Visiting friend: b'Viola Quenzer'
Found 0 mutual friends
Visiting friend: b'Adam Keel'
Found 0 mutual friends
Visiting friend: b'Stefan Glantschnig'
Found 0 mutual friends
Visiting friend: b'Isa Iskristaller'
Found 0 mutual friends
Visiting friend: b'Raphal Dunant'
Found 0 mutual friends
Visiting friend: b'Melissa Steuri'
Found 0 mutual friends
Visiting friend: b'Angelo Zehr'
Found 0 mutual friends
Visiting friend: b'Cornelia Jost'
Found 0 mutual friends
Visiting friend: b'Oliver Fuchs'
Found 0 mutual friends
Visiting friend: b'Kerstin Hasse'
Found 0 mutual friends
Visiting friend: b'Fabio Vonarburg'
Found 0 mutual friends
Visiting friend: b'Simone Honegger'
Found 0 mutual friends
Visiting friend: b'Lea Hartmann'
Found 0 mutual friends
Visiting friend: b'Lisa Stutz'
Found 0 mutual friends
Visiting 

Found 0 mutual friends
Visiting friend: b'Janosch Trhler'
Found 0 mutual friends
Visiting friend: b'Lenard Baum'
Found 0 mutual friends
Visiting friend: b'Marie Ceriani'
Found 0 mutual friends
Visiting friend: b'Mario Tipura'
Found 0 mutual friends
Visiting friend: b'Jakob Libertin'
Found 0 mutual friends
Visiting friend: b'Flurina Dnki'
Found 0 mutual friends
Visiting friend: b'Theresa Stiegler'
Found 0 mutual friends
Visiting friend: b'Graldine Bugnard'
Found 0 mutual friends
Visiting friend: b'Dieter Fahrer'
Found 0 mutual friends
Visiting friend: b'Lukas Wegmller'
Found 0 mutual friends
Visiting friend: b'Jason Dimitratos'
Found 0 mutual friends
Visiting friend: b'Jrme Favre Bundeshaus'
Found 0 mutual friends
Visiting friend: b'Dominik Meienberg'
Found 0 mutual friends
Visiting friend: b'Jannis Strauss'
Found 0 mutual friends
Visiting friend: b'Luca Rena Fuchs'
Found 0 mutual friends
Visiting friend: b'Martina Polek'
Found 0 mutual friends
Visiting friend: b'Tobias Tscherrig'
Found

Found 0 mutual friends
Visiting friend: b'Sally Knzler'
Found 0 mutual friends
Visiting friend: b'Yves Sinka'
Found 0 mutual friends
Visiting friend: b'Charlotte Theile'
Found 0 mutual friends
Visiting friend: b'Ramona Rehli'
Found 0 mutual friends
Visiting friend: b'Nicolas NIci Senn'
Found 0 mutual friends
Visiting friend: b'Nadine Annen'
Found 0 mutual friends
Visiting friend: b'Ingo Walder'
Found 0 mutual friends
Visiting friend: b'Moritz Meier'
Found 0 mutual friends
Visiting friend: b'Nicola Imfeld'
Found 0 mutual friends
Visiting friend: b'Merve nsal'
Found 0 mutual friends
Visiting friend: b'Ramuel Sauber'
Found 0 mutual friends
Visiting friend: b'Kristijan Jovanovi'
Found 0 mutual friends
Visiting friend: b'Constantin Seibt'
Found 0 mutual friends
Visiting friend: b'Valentin Danzi'
Found 0 mutual friends
Visiting friend: b'Marco Paganini'
Found 0 mutual friends
Visiting friend: b'Jana Tlos'
Found 0 mutual friends
Visiting friend: b'Severin Schweizer'
Found 0 mutual friends
Vis

KeyboardInterrupt: 

In [24]:
import networkx as nx

def create_user_network():
    friendslist = []
    edgelist = []
    
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"],row["Friend Name"]])        

    with open('network.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                edgelist.append([row["From"],row["To"]])
    
    G  = nx.Graph(name="Friends")
                            
    for friend in friendslist:
        G.add_node(friend[0],label=friend[1], size=1)

    for edge in edgelist:
        G.add_edge(edge[0],edge[1],weight=1)
   
    nx.write_gexf(G,"Friends.gexf")

In [25]:
create_user_network()